# exract

In [1]:
import pandas as pd
import numpy as np

In [3]:
df1 = pd.read_csv('./sensors_temp_data.csv')
print(df1.head())

  result  table                     _start                             _stop  \
0   mean      0  1970-01-01 00:00:00+00:00  2025-10-22 07:48:40.671758+00:00   
1   mean      0  1970-01-01 00:00:00+00:00  2025-10-22 07:48:40.671758+00:00   
2   mean      0  1970-01-01 00:00:00+00:00  2025-10-22 07:48:40.671758+00:00   
3   mean      0  1970-01-01 00:00:00+00:00  2025-10-22 07:48:40.671758+00:00   
4   mean      0  1970-01-01 00:00:00+00:00  2025-10-22 07:48:40.671758+00:00   

                       _time     _value _field _measurement          host  \
0  2025-10-16 11:15:00+00:00  24.800000   temp     cucumber  f199a2094793   
1  2025-10-17 02:10:00+00:00  24.857143   temp     cucumber  f199a2094793   
2  2025-10-17 02:15:00+00:00  24.946667   temp     cucumber  f199a2094793   
3  2025-10-17 02:20:00+00:00  24.920000   temp     cucumber  f199a2094793   
4  2025-10-17 02:25:00+00:00  24.724138   temp     cucumber  f199a2094793   

           sensor_id                               topic

# เปลี่ยนชื่อ column

In [5]:
import pandas as pd
from datetime import datetime
import pytz

# Read CSV
df1 = pd.read_csv('./sensors_temp_data.csv')

# Create df2 with selected columns
df2 = df1[['_time', 'sensor_id', '_value']].copy()

# Rename columns
df2.columns = ['timestamp', 'mac_address', 'temperature']

# Convert timestamp to datetime with ISO8601 format and then to GMT+7 (Thailand)
df2['timestamp'] = pd.to_datetime(df2['timestamp'], format='ISO8601').dt.tz_convert('Asia/Bangkok')

# Save to CSV
df2.to_csv('processed_sensors_data.csv', index=False)

# Print first few rows
print(df2.head())

                  timestamp        mac_address  temperature
0 2025-10-16 18:15:00+07:00  60:55:F9:EC:C3:D6    24.800000
1 2025-10-17 09:10:00+07:00  60:55:F9:EC:C3:D6    24.857143
2 2025-10-17 09:15:00+07:00  60:55:F9:EC:C3:D6    24.946667
3 2025-10-17 09:20:00+07:00  60:55:F9:EC:C3:D6    24.920000
4 2025-10-17 09:25:00+07:00  60:55:F9:EC:C3:D6    24.724138


# grouping ข้อมูล by mac address

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Ensure output directory exists
output_dir = './img_out_temp'
os.makedirs(output_dir, exist_ok=True)

# Read CSV
df2 = pd.read_csv('processed_sensors_data.csv')

# Print unique MAC addresses
print("Unique MAC Addresses:")
print(df2['mac_address'].unique())

# Create df3: Group by mac_address and calculate mean temperature
df3 = df2.groupby('mac_address')['temperature'].mean().reset_index()
print("\nDF3: Mean Temperature by MAC Address")
print(df3)

# Convert timestamp to datetime with ISO8601 format and extract date for daily grouping
df2['timestamp'] = pd.to_datetime(df2['timestamp'], format='ISO8601')
df2['date'] = df2['timestamp'].dt.date

# Group by mac_address and date, calculate mean temperature
daily_data = df2.groupby(['mac_address', 'date'])['temperature'].mean().reset_index()

# Plot temperature for each MAC address, separated by date
mac_addresses = daily_data['mac_address'].unique()

for mac in mac_addresses:
    # Replace invalid characters in MAC address for filename
    safe_mac = mac.replace(':', '_').replace(',', '_')
    mac_data = daily_data[daily_data['mac_address'] == mac]
    plt.figure(figsize=(10, 5))
    plt.plot(mac_data['date'], mac_data['temperature'], marker='o', label=f'MAC: {mac}')
    plt.title(f'Average Daily Temperature for {mac}')
    plt.xlabel('Date')
    plt.ylabel('Temperature (°C)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/temperature_plot_{safe_mac}.png')
    plt.close()

Unique MAC Addresses:
['60:55:F9:EC:C3:D6' '60:55:F9:EC:D4:4E' '60:55:F9:ED:22:2C'
 '60:55:F9:ED:36:E8' '60:55:F9:ED:92:00' '60:55:F9:ED:D2:18'
 '60:55:F9:ED:F5:DC' '68:67:25:2D:02:6E' '7C:DF:A1:00:AA:66'
 '7C:DF:A1:00:AD:6E' 'SENSOR08,' 'cucumber_4']

DF3: Mean Temperature by MAC Address
          mac_address  temperature
0   60:55:F9:EC:C3:D6    24.748179
1   60:55:F9:EC:D4:4E    25.725963
2   60:55:F9:ED:22:2C    24.433299
3   60:55:F9:ED:36:E8    25.071865
4   60:55:F9:ED:92:00    24.800406
5   60:55:F9:ED:D2:18    24.406814
6   60:55:F9:ED:F5:DC    25.649304
7   68:67:25:2D:02:6E    24.148925
8   7C:DF:A1:00:AA:66    25.118975
9   7C:DF:A1:00:AD:6E    24.871922
10          SENSOR08,    24.900000
11         cucumber_4    25.666667


# คำอธิบายโค้ด (สั้น ๆ)

โค้ดนี้ทำการวิเคราะห์ข้อมูลอุณหภูมิจากไฟล์ `processed_sensors_data.csv`:
1. อ่าน CSV และแสดง MAC address ที่ไม่ซ้ำ.
2. สร้าง `df3` โดยคำนวณอุณหภูมิเฉลี่ยต่อ MAC address.
3. แปลง timestamp เป็นวันที่ (GMT+7) และจัดกลุ่มข้อมูลตาม MAC address และวันที่.
4. สร้างกราฟอุณหภูมิเฉลี่ยรายวันสำหรับแต่ละ MAC address.
5. แก้ปัญหา Invalid filename โดยแปลง `:` และ `,` ใน MAC address เป็น `_` และบันทึกกราฟลงโฟลเดอร์ `img_out_temp` เป็น PNG.